##### Обновленная модель. Добавлен механизм детектирования аномалий. Реализованы разные алгоритмы.
Используется только один признак, т.е. волатильность

In [1]:
import ta
import math
import pickle
import numpy as np
import pandas as pd
import pandas_profiling
from tqdm import tqdm
from copy import deepcopy
from sklearn.mixture import GaussianMixture

In [2]:
import cufflinks as cf
import plotly.offline
import plotly.express as px
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [4]:
path = '/home/vlad/HDD/GIT/CryptoLp DataScience/DataSets/Разные бары/'
file = '2019-2020_Bars_700T'
df_data = pd.read_csv(path+file+'.csv')
df_data = df_data.dropna()
df_data.index = range(len(df_data))

In [5]:
df_data.tail(3)

,datetime,open,low,high,close,volume
585697,2021-01-10 23:57:36.495287,38428.0,38300.0,38462.5,38300.0,4430463
585698,2021-01-10 23:59:30.593566,38300.0,38250.0,38357.5,38264.5,8412985
585699,2021-01-10 23:59:59.886955,38264.5,38225.0,38279.5,38225.0,5901463


In [6]:
# Вычисление стандартной волатильности
def calculation_volatility_STD(df_data_close_price:pd.Series,
                               win_size):
    volatility_STD = df_data_close_price.pct_change().rolling(win_size).std()*1000
    return volatility_STD

In [7]:
high = df_data['high']
low = df_data['low']
close = df_data['close']
volume = df_data['volume']

In [8]:
# %%time
df_data['volatility'] = calculation_volatility_STD(df_data_close_price=df_data['close'], win_size=1000)

In [9]:
df_data = df_data.dropna()
df_data.index = range(len(df_data))

In [10]:
df_data.head(3)

,datetime,open,low,high,close,volume,volatility
0,2019-01-02 20:31:01.706620,3841.0,3841.0,3845.5,3845.5,946234,1.269966
1,2019-01-02 20:32:12.833091,3845.5,3845.5,3847.0,3846.0,1511543,1.269043
2,2019-01-02 20:34:47.248511,3846.0,3844.5,3848.0,3848.0,2360445,1.268973


In [11]:
anomalies = list()

### Подход 1. Статистический

In [12]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler

# volatility = df_data[['volatility']]
# scaler = MinMaxScaler()
# scaled_volat = scaler.fit_transform(volatility)
# df_data['scaled_vol'] = scaled_volat 
# vol_std = np.std(scaled_volat)
# vol_mean = np.mean(scaled_volat)
# anomaly_cut_off = vol_std * 3

# lower_limit = vol_mean - anomaly_cut_off 
# upper_limit = vol_mean + anomaly_cut_off

# for outlier in scaled_volat:
#     if outlier > upper_limit or outlier < lower_limit:
#         anomalies.append(-1)
#     else:
#         anomalies.append(1)
# df_data['outlier_STD'] = anomalies

#### Новая разметка аномалий

In [12]:
# %%time
# anomalies = list()

# lbord = df_data['scaled_vol'].quantile(0.01)
# rbord = df_data['scaled_vol'].quantile(0.99)

# scaled_volat = df_data[['scaled_vol']].values
# for outlier in scaled_volat:
#     if outlier > rbord or outlier < lbord:
#         anomalies.append(-1)
#     else:
#         anomalies.append(1)
# df_data['outlier_Stat'] = anomalies

In [13]:
# # Построение графиков
# def show_plot_plotly(df_data:pd.DataFrame, left_border,right_border, x_,y_, color_:str, title_:str,             
#                     ):  
#     fig = px.scatter(df_data[left_border:right_border], 
#                      x=x_, y=y_, color=color_, 
#                      #color_discrete_sequence=['dodgerblue','mediumturquoise','limegreen','tomato','darkorchid',],
#                      title=title_
#                     )    
#     return fig

# df_data['outlier_STD'] = df_data['outlier_STD'].map({1:'clean',-1:'anomaly',})
# df_data['outlier_Stat'] = df_data['outlier_Stat'].map({1:'clean',-1:'anomaly',})

# fig = show_plot_plotly(df_data=df_data, left_border = 0, right_border=len(df_data),
#                        x_='datetime', y_='close', color_='outlier_STD', title_='outlier_STD',
#                       )
# plotly.offline.plot(fig,filename='/home/brainiac/Vlad/Git Files/CryptoLp DataScience/DataSets/Bars/outlier_STD.html')

# fig = show_plot_plotly(df_data=df_data, left_border = 0, right_border=len(df_data),
#                        x_='datetime', y_='close', color_='outlier_Stat', title_='outlier_Stat',
#                       )
# plotly.offline.plot(fig,filename='/home/brainiac/Vlad/Git Files/CryptoLp DataScience/DataSets/Bars/outlier_Stat.html')

### Подход 2. Кластеризация

In [14]:
# %%time
# from sklearn.cluster import DBSCAN

# outlier_detection = DBSCAN(min_samples = 2)
# clusters = outlier_detection.fit_predict(volatility)

In [15]:
#Упал по оперативке

### Подход 3. Изоляция леса

In [16]:
df_data.head(3)

,datetime,open,low,high,close,volume,volatility
0,2019-01-02 20:31:01.706620,3841.0,3841.0,3845.5,3845.5,946234,1.269966
1,2019-01-02 20:32:12.833091,3845.5,3845.5,3847.0,3846.0,1511543,1.269043
2,2019-01-02 20:34:47.248511,3846.0,3844.5,3848.0,3848.0,2360445,1.268973


In [17]:
%%time
from sklearn.ensemble import IsolationForest
clf = IsolationForest(max_samples=100, random_state = 1, contamination= 'auto')

volatility = df_data[['volatility']]
anomalies = clf.fit_predict(volatility)

CPU times: user 5.16 s, sys: 836 ms, total: 6 s
Wall time: 6.02 s


In [18]:
df_data['outlier_IsoForest'] = anomalies

In [19]:
df_data['outlier_IsoForest'] = df_data['outlier_IsoForest'].map({1:'clean',-1:'anomaly',})

In [20]:
df_data.head(2)

,datetime,open,low,high,close,volume,volatility,outlier_IsoForest
0,2019-01-02 20:31:01.706620,3841.0,3841.0,3845.5,3845.5,946234,1.269966,clean
1,2019-01-02 20:32:12.833091,3845.5,3845.5,3847.0,3846.0,1511543,1.269043,clean


## Выделение режимов

In [21]:
# Кластеризация
def clustering_GaussianHMM(df_data:pd.DataFrame,
                           n_components_:int,
                          ):
    model = GaussianMixture(n_components=n_components_, covariance_type="full")
    model.fit(df_data)
    marked_regimes = model.predict(df_data)
    return marked_regimes

In [22]:
df_data_clear = df_data[df_data['outlier_IsoForest']=='clean']
df_data_outlier = df_data[df_data['outlier_IsoForest']=='anomaly']

In [23]:
X = df_data_clear[['volatility']]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(X[['volatility']])
X['volatility'] = scaled*10

In [24]:
marked_regimes = clustering_GaussianHMM(df_data = X, n_components_= 4)

In [25]:
len(marked_regimes)
len(df_data_clear)

460563

460563

#### Формирование результата

In [26]:
df_data_clear['regimes'] = marked_regimes
df_data_outlier['regimes'] = -1

In [27]:
df_data_clear['regimes'].unique()

array([3, 2, 1, 0])

In [28]:
df_data_clear.head(1)
df_data_outlier.head(1)

,datetime,open,low,high,close,volume,volatility,outlier_IsoForest,regimes
0,2019-01-02 20:31:01.706620,3841.0,3841.0,3845.5,3845.5,946234,1.269966,clean,3


,datetime,open,low,high,close,volume,volatility,outlier_IsoForest,regimes
14245,2019-02-02 06:29:16.438661,3417.5,3416.0,3417.5,3416.0,1818677,0.64791,anomaly,-1


In [29]:
df_result = pd.concat([df_data_clear, df_data_outlier])
df_result = df_result.sort_values('datetime')
df_result.index = range(len(df_result))

### Построение графика

In [30]:
df_result.head(2)
df_result['regimes'].unique()

,datetime,open,low,high,close,volume,volatility,outlier_IsoForest,regimes
0,2019-01-02 20:31:01.706620,3841.0,3841.0,3845.5,3845.5,946234,1.269966,clean,3
1,2019-01-02 20:32:12.833091,3845.5,3845.5,3847.0,3846.0,1511543,1.269043,clean,3


array([ 3,  2,  1,  0, -1])

In [31]:
# Построение графиков
def show_plot_plotly(df_data:pd.DataFrame,
                     left_border, right_border,
                     x_,y_, color_:str, title_:str,             
             ):  
    fig = px.scatter(df_data[left_border:right_border], 
                     x=x_, y=y_, color=color_, 
                     #color_discrete_sequence=['dodgerblue','mediumturquoise','limegreen','tomato','darkorchid',],
                     title=title_
                    )    
    return fig

### Сортировка режимов

In [32]:
#df_result['outlier_STD'] = df_result['outlier_STD'].map({-1:'-1', 1:'1',})
df_result['regimes'] = df_result['regimes'].map({0:'0', 1:'1', 2:'2', 3:'3', -1:'-1'})

In [33]:
regime_0 = df_result[df_result['regimes'] == '0']
regime_1 = df_result[df_result['regimes'] == '1']
regime_2 = df_result[df_result['regimes'] == '2']
regime_3 = df_result[df_result['regimes'] == '3']

In [34]:
(regime_0['volatility'].min()+regime_0['volatility'].mean()+regime_0['volatility'].max(), 0)
(regime_1['volatility'].min()+regime_1['volatility'].mean()+regime_1['volatility'].max(), 1)
(regime_2['volatility'].min()+regime_2['volatility'].mean()+regime_2['volatility'].max(), 2)
(regime_3['volatility'].min()+regime_3['volatility'].mean()+regime_3['volatility'].max(), 3)

(2.2969073100316235, 0)

(4.500750256842466, 1)

(3.0137245513668827, 2)

(3.7175360285826864, 3)

In [35]:
df_result['regimes'] = df_result['regimes'].map({'0':'regime0', 
                                                 '2':'regime1',
                                                 '3':'regime2', 
                                                 '1':'regime3', 
                                                 '-1':'Anomaly',
                                                })

In [36]:
df_result.head()

,datetime,open,low,high,close,volume,volatility,outlier_IsoForest,regimes
0,2019-01-02 20:31:01.706620,3841.0,3841.0,3845.5,3845.5,946234,1.269966,clean,regime2
1,2019-01-02 20:32:12.833091,3845.5,3845.5,3847.0,3846.0,1511543,1.269043,clean,regime2
2,2019-01-02 20:34:47.248511,3846.0,3844.5,3848.0,3848.0,2360445,1.268973,clean,regime2
3,2019-01-02 20:35:07.507625,3848.0,3848.0,3852.0,3851.0,1272379,1.269183,clean,regime2
4,2019-01-02 20:35:20.780445,3851.0,3851.0,3857.0,3857.0,811095,1.269099,clean,regime2


In [38]:
fig = show_plot_plotly(df_data=df_result,
                       left_border = 0,
                       right_border=len(df_result),
                       x_='datetime',
                       y_='close',
                       color_='regimes',
                       title_='regimes',
                      )
plotly.offline.plot(fig,filename='3.html')

'3.html'

In [39]:
df_result.head()

,datetime,open,low,high,close,volume,volatility,outlier_IsoForest,regimes
0,2019-01-02 20:31:01.706620,3841.0,3841.0,3845.5,3845.5,946234,1.269966,clean,regime2
1,2019-01-02 20:32:12.833091,3845.5,3845.5,3847.0,3846.0,1511543,1.269043,clean,regime2
2,2019-01-02 20:34:47.248511,3846.0,3844.5,3848.0,3848.0,2360445,1.268973,clean,regime2
3,2019-01-02 20:35:07.507625,3848.0,3848.0,3852.0,3851.0,1272379,1.269183,clean,regime2
4,2019-01-02 20:35:20.780445,3851.0,3851.0,3857.0,3857.0,811095,1.269099,clean,regime2


In [40]:
df_result[(df_result['datetime']>="2020-03-01") & 
          (df_result['datetime']<"2021-02-01")][['datetime','regimes']].to_csv('/home/vlad/HDD/GIT/Файлы для бэктестинга/test.csv',
                                                                               index=False)